In [1]:
imagesFolder = '/home/ImageNetPart/'
%tensorflow_version 2.x
# !pip uninstall imageio
# !pip install imageio

def initColab_Full():
    from google.colab import drive
    # !set | more
    drive.mount('/home/gdrive')
    %cd "/home/gdrive/My Drive/Visualiz_Zeiler"

    !pip uninstall keras
    !pip install keras
    # !pip uninstall tensorflow
    # !pip install tensorflow==2.0

    !df -hm | grep overlay
    # !unzip VKIImageNetPart_TrainTestDivided.zip -d $imagesFolder >/dev/null
    # !df -hm
    %mv $imagesFolder/Test $imagesFolder/test
    %mv $imagesFolder/Train $imagesFolder/train
    
def initCurRuntime():
    %cd "/home/gdrive/My Drive/Visualiz_Zeiler"
    # %rm *\(1\)*
    %ls *Lo* -l
    # %pwd
    # %rm *.h5

    %matplotlib inline
    %mkdir -p "Logs2/src2"
    # %ls .
    %cp -f ./*py* Logs2/src2/

# initColab_Full()    # Mounts Google drive and unzips images
initCurRuntime()

from VisColabUtils import *


TensorFlow 2.x selected.
/home/gdrive/My Drive/Visualiz_Zeiler
total 4
drwx------ 2 root root 4096 Jan  1 12:39 src2/
cp: -r not specified; omitting directory './__pycache__'
2.1.0-rc1


In [0]:
# !pip uninstall matplotlib python-qt imgaug
# !python -m pip install --upgrade pip
# !pip install matplotlib
# !pip install python-qt4

In [3]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
@interact   # Doesn't work now at VKI's GeForce, works at CoLab
def f(x=5):
    image = controlObj.imageDataset.getImage(x, 'cropped')
    imshow(image);
    return x
# interact(f, x=10);

imageNum = 8
# image = controlObj.imageDataset.getImage(imageNum, 'cropped')
# imshow(image);
%pip freeze >req.txt

interactive(children=(IntSlider(value=5, description='x', max=15, min=-5), Output()), _dom_classes=('widget-in…

In [4]:
epochNum = -3
print(tf.__version__)
print(controlObj.getSelectedEpochNum())
layerName = 'conv_11'
activations, drawMode, stdData = controlObj.getActivationsData(epochNum, imageNum, layerName)
actImage = layoutLayersToOneImage(np.sqrt(activations), colCount, margin)
print(activations.shape, actImage.shape)
imshow(actImage);

seconds from start (passed microseconds)
2.1.0-rc1
3.057 (    77.00) -2
Model 9 2


Using TensorFlow backend.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 227, 227, 3)  0                                            
__________________________________________________________________________________________________
conv_11 (Conv2D)                (None, 74, 74, 96)   18528       input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 74, 74, 32)   0           conv_11[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 74, 74, 32)   0           conv_11[0][0]                    
____________________________________________________________________________________________

In [0]:
for i in range(0):
    activations = controlObj.netWrapper.getImageActivations('dense_2', i, epochNum)
    print(activations);

def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [192, 192])
  image /= 255.0  # normalize to [0,1] range
  return image
def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  return preprocess_image(image)
def load_and_preprocess_image_with_label(path, label):
  print(path)
  image = tf.io.read_file(path)
  print(image, label)
  return preprocess_image(image), label

all_image_paths = ["ImageNetPart/American crow/1003355_8d39d66686.jpg", 
                   "ImageNetPart/barn owl/101715431_19d0b37108.jpg"]
labels = ['crow', 'owl']
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=1)
# for n, image in enumerate(load_image_ds.take(7)):
#     print(image)
    
label_ds = tf.data.Dataset.from_tensor_slices(labels)
ds = tf.data.Dataset.zip((image_ds, label_ds))
# ds = tf.data.Dataset.zip((path_ds, label_ds))
# for n, v in enumerate(ds.take(7)):
#     print(v)
ds = ds.repeat()
# ds = ds.map(load_and_preprocess_image_with_label, num_parallel_calls=1)

In [6]:
# Internal dataset's check
import tensorflow as tf
print(tf.executing_eagerly())
ds = controlObj.imageDataset.getTfDataset('test') 
# print(ds)
plt.figure(figsize=(8,8))
for n, val in enumerate(ds.take(3)):
    # ??val
    # print(val)
    imageNum = val[0].numpy()
    # classInd = val[1].numpy()
    classInd = val[1].numpy()
#     if imageNum % 1000 == 0:
#         print(val)
#     continue
    className = controlObj.imageDataset.getClassNameLabel(classInd) #.decode('ascii')
#     print(image.numpy().shape)
    ax = plt.subplot(4,4,n+1)
    plt.text(1.1, 0.5, '%s (%s)' % (className, str(classInd)),  
          transform = ax.transAxes)
    image = controlObj.imageDataset.getImage(imageNum, 'cropped', 'test')
    plt.imshow(image / 255.0)
    #   plt.legend( bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.show()

5.468 ( 60951.00) True


In [7]:
# Final (batched) datasets' check
# controlObj.netWrapper._initMainNet()
# print(controlObj.netWrapper)
trainImageNums = np.arange(1, controlObj.imageDataset.getImageCount('train') + 1)
testImageNums = np.arange(1, controlObj.imageDataset.getImageCount('test') + 1)
tfTrainDataset, tfTestDataset = controlObj.netWrapper.net._getTfDatasets(
                trainImageNums, testImageNums, 1000)
# tfDataset = tf.compat.v1.data.make_one_shot_iterator(tfDataset)
# ??tfDataset

print(controlObj.imageDataset.getImageCount('train'))
if 1:
    plt.figure(figsize=(8,8))
    for n, val in enumerate(tfTrainDataset.take(2)):
        print(controlObj.netWrapper.getCacheStatusInfo(True))
        images = val[0].numpy()
    #     ??images
        print(images.shape)
        matrixImage = layoutLayersToOneImage(images, colCount, margin)
        imshow(matrixImage);
    #     imshow(images[0])
        plt.show()

if 0:
 for n, val in enumerate(tfDataset.take(0)):
  # ??val
  image = val[0][0].numpy()
  # classInd = val[1].numpy()
  # className = controlObj.imageDataset.getClassNameByInd(classInd) #.decode('ascii')
  classVec = val[1][0].numpy()
  print(image.shape)
  ax = plt.subplot(4,4,n+1)
  plt.text(1.1, 0.5, str(classVec),  
          transform = ax.transAxes)
  plt.imshow(image /255.0)
#   plt.legend( bbox_to_anchor=(1.05, 1), loc='upper left')
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  plt.show()


5.719 (250752.00) 12402
9.944 (4225130.00) acts: 2.01 MBs, images: 132 object(s), 0 + 0 in LRU lists, 24.55 MBs
9.957 ( 12763.00) (64, 227, 227, 3)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


10.449 (491824.00) acts: 2.01 MBs, images: 142 object(s), 0 + 0 in LRU lists, 26.42 MBs
10.475 ( 25834.00) (64, 227, 227, 3)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


if 1:
    imList = enumerate(tfDataset.take(6))
    #??imList
#     print(imList[0].shape())
else:
    v = next(tfDataset)
    imList = [im for im in v[0][:6]]
matrixImage = layoutLayersToOneImage(np.stack(imList), 3, margin)
imshow(matrixImage);
print(v[1][:6])

In [8]:
controlObj.netWrapper.imageCache.maxMemory = 4 << 30
controlObj.onDoItersPressed(1)

18.861 (8386397.00) Cur control object epoch 0, wrapper - -2
18.890 ( 28794.00) Learning rate switched to 0.000100
Epoch 1/1
 - 64s - loss: 0.0535 - accuracy: 0.0484 - val_loss: 0.0400 - val_accuracy: 0.0625
142.576 (123685560.00) acts: 2.01 MBs, images: 4571 object(s), 162 + 0 in LRU lists, 808.77 MBs
142.577 (  1213.00) Error in saveState: Unable to create file (unable to open file: name = 'QtLogs/ImagesWeights_Epoch1.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)
142.577 (   693.00) Epoch 1: loss 0.05354, acc 0.0484, last 1 epochs: 66.7646 s
142.578 (   978.00) Epoch 1 finished
142.580 (  1100.00) Epoch 1: loss 0.05354, acc 0.0484, last 1 epochs: 66.7646 s


In [9]:
print(controlObj.netWrapper.getCacheStatusInfo(True))


142.584 (  4451.00) acts: 2.01 MBs, images: 4571 object(s), 162 + 0 in LRU lists, 808.77 MBs


In [0]:
controlObj.onDoItersPressed(1000)

142.590 (  6351.00) Cur control object epoch 1, wrapper - -2
Epoch 2/2


In [0]:
controlObj.onDoItersPressed(1000000)

In [0]:
controlObj.onDoItersPressed(1000000)

In [0]:

plt.figure(figsize=(20,10))
columns = 5
for i in range(3):
    imshow(activations[i],cmap='Greys_r')   # Displays only one
#display(activations[1])

In [0]:
def showImagesMatrix(data, col=2):
    fig = figure( figsize=(600, 300))
    number_of_files = len(data)
    print(number_of_files)
    row = number_of_files/col
    if (number_of_files%col != 0):
        row += 1
    for i in range(number_of_files):
        a=fig.add_subplot(row,col,i+1)
        # image = imread(mypath+'/'+list_of_files[i])
        axis('off')
        print('Image %d drawn' % i)
    # Very slow at the end
        
showImagesMatrix(activations[1:8])

In [0]:
for i in np.random.permutation(600)[:4]:
    count = count + 1
    plt.subplot(1, sample_size, count)
    plt.axhline('')
    plt.axvline('')
    plt.text(x=10, y=-10, s=y_train[i], fontsize=18)
    plt.imshow(X_train[i].reshape(28, 28), cmap=plt.cm.Greys)

In [0]:
im = Image(filename="image/000644.jpg", width=100, height=100)
im.shape()

In [0]:
![title](images/000644.png)

In [0]:
from IPython.display import Image          # Displaying of multiple images. To check
from IPython.display import display
x = Image(filename='1.png') 
y = Image(filename='2.png') 
display(x, y)

In [0]:
out = widgets.Output(layout={'border': '1px solid black'})
with out:
    display(widgets.IntSlider())